In [23]:
import pandas as pd
import numpy as np
import geopy.distance

In [24]:
dfPlaces = pd.read_csv('../data/externalData/placesOfInterest.csv', encoding='utf-8')
dfTrain = pd.read_csv('../data/TRAIN_TEST_corrected2/train_corrected2.csv')

In [25]:
dfTrain.loc[dfTrain.id.str.contains('1771e4252915a911b8eb4964cde2790d5623bd33')]

,id,year_created,month_created,day_created,distance_to_obelisco_mtrs,lat,lon,surface_total_in_m2,rooms,price_usd,PH,apartment,house,store
0,1771e4252915a911b8eb4964cde2790d5623bd33,2013.0,8.0,28.0,16861.066565,-34.600116,-58.565334,62.0,3.0,130650.48,0.0,1.0,0.0,0.0


In [26]:
dfPlaces.tail(10)

,place_id,lat,lon,name,topic,centroid
2389,ChIJR_mVyca1vJURBWmPMGrA41c,-34.570247,-58.444337,Olleros,SUBTES,CABA
2390,ChIJ9YiBmoLLvJURTgPWJPEA1LE,-34.659502,-58.418041,Espacio Verde Ezequiel Demonty,PARQUES,SUR
2391,ChIJ7RNuPKS1vJURiR7kLC7sLro,-34.563292,-58.424077,Cnel. Jordan C. Wysocki Park,PARQUES,OESTE
2392,ChIJ0edzkJ7JvJURV50M-djr97E,-34.644294,-58.476915,Calesita Parque Nicolás Avellaneda,PARQUES,CABA
2393,ChIJRzAK7Zy2vJURHmHHuYWAJv4,-34.550454,-58.467112,Juana Azurduy,METROBUS,CABA
2394,ChIJp6o51pjKvJURJHk8cSWRqpc,-34.588154,-58.398162,Las Heras,SUBTES,OESTE
2395,ChIJkyFHrHe1vJURd0a3ENmtgyg,-34.574960,-58.415052,Monumento a La Carta Magna y las Cuatro Region...,TURISMO,NORTE
2396,ChIJC-COMCjLvJURvGt0t-UtDkY,-34.618231,-58.381580,Independencia,SUBTES,OESTE
2397,ChIJwUyDAjM1o5URhqe69AlgVfM,-34.599536,-58.397953,School of Medicine,SUBTES,NORTE
2398,ChIJ3Y0AxjM1o5URQUk53zruw3o,-34.600735,-58.366840,Tourist Assistance Center - Puerto Madero,TURISMO,NORTE


In [27]:
def _entornoScore(puntaje, d, dmax) :
    
    if (d > dmax) :
        return 0
    
    exp = 3/float(2)
    return puntaje * (1 - (d/float(dmax))**exp )

In [28]:
def entornoScore(lat, lon, df, puntajes) :
    
    LAT, LON, TOPIC = (1, 2, 4)
    score = 1   # Los scores comienzan en 1
    
    for row in df.as_matrix() :
        topic = row[TOPIC]
        lat_ = row[LAT]
        lon_ = row[LON]
        
        pje = puntajes[topic]['puntaje']
        dist_max = puntajes[topic]['dist_max_mts']
        
        dist_a_pto_interes = geopy.distance.vincenty( (lat_,lon_), (lat, lon) ).m
        score += _entornoScore(pje, dist_a_pto_interes, dist_max)
    
    return score

In [29]:
def makeScoresForIds(df) :    
    
    # PUNTAJES:
    puntajes = { 'SUBTES'    : { 'puntaje':80,  'dist_max_mts':500 }, 
                 'SHOPPINGS' : { 'puntaje':100, 'dist_max_mts':900 }, 
                 'METROBUS'  : { 'puntaje':40,  'dist_max_mts':300 }, 
                 'TRENES'    : { 'puntaje':65,  'dist_max_mts':500 }, 
                 'PARQUES'   : { 'puntaje':30,  'dist_max_mts':300 }, 
                 'TURISMO'   : { 'puntaje':20,  'dist_max_mts':400 }, 
                 'SEGURIDAD' : { 'puntaje':45,  'dist_max_mts':250 }
               }
    
    # Inicializo todos los scores en NaN
    id_scores = dict([ (row, np.nan) for row in df.id ])
    ID, lat, lon = (0, 5, 6)
    
    # Itero el dataframe grande
    for row in df.as_matrix() :
        id_scores[ row[ID] ] = entornoScore(row[lat], row[lon], dfPlaces, puntajes)
    
    return pd.DataFrame(id_scores.items(), columns=['id', 'entorno_score'])

In [31]:
dfScores = makeScoresForIds(dfTrain)

In [34]:
dfScores.head(8)

,id,entorno_score
0,e3b65ed18efd6b77af3ec6c9c56ef2071b622fc0,1.000000
1,1771e4252915a911b8eb4964cde2790d5623bd33,1.000000
2,5f97b70603ef4e0d8196ad477dfd41eebc402ad1,1.000000
3,005481f73ee4472d5c42f326537cc319af2a5f40,741.478746
4,ba2b56885dbb1ad9d24d6dd5f92d4b04d66d1f5b,114.738898
5,486c44c6ae60cc40aa7aac085bf20a83a3e68e0e,1.000000
6,3310de102f192c4a03a415e3f648799caef02da7,242.680546
7,d63560d1f8f797a6d412814a27ec570beafb9d90,574.505229


In [33]:
dfTrain.loc[dfTrain.id.str.contains('005481f73ee4472d5c42f326537cc319af2a5f40')]

,id,year_created,month_created,day_created,distance_to_obelisco_mtrs,lat,lon,surface_total_in_m2,rooms,price_usd,PH,apartment,house,store
4,005481f73ee4472d5c42f326537cc319af2a5f40,2013.0,8.0,28.0,7561.524039,-34.626409,-58.459324,40.0,1.0,110714.41,0.0,1.0,0.0,0.0


In [ ]:
dfPlaces.groupby(by='topic').count().loc[:, ('place_id')]